## Problem 3: How many people live near shopping centers? (8 points)

In the last step of this analysis, use a *spatial join* to relate data from a population grid data set to the buffer layer created in *problem 2* to find out how many people live in all population grid cells that are **within** 1.5 km distance from each shopping centre. 

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS).


*Feel free to divide your solution into more codeblocks than prepared! Remember to add comments to your code :)*

### a) Load the population grid data set and the buffer geometries

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS). Load the data into a `GeoDataFrame` called `population_grid`.

(optional) If you want, discard unneeded columns and translate the remaining column names from Finnish to English.

In [1]:
# ADD YOUR OWN CODE HERE

import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

import geopandas as gpd
import pandas as pd

population_grid = gpd.read_file(
    (
        "https://kartta.hsy.fi/geoserver/wfs"
        "?service=wfs"
        "&version=2.0.0"
        "&request=GetFeature"
        "&typeName=asuminen_ja_maankaytto:Vaestotietoruudukko_2020"
        "&srsName=EPSG:3879"
    ),
)
population_grid.crs = crs="EPSG:3879"  # for WFS data, the CRS needs to be specified manually

In [2]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import geopandas
import pyproj

assert isinstance(population_grid, geopandas.GeoDataFrame)
assert population_grid.crs == pyproj.CRS("EPSG:3879")



Load the buffers computed in *problem 2* into a `GeoDataFrame` called `shopping_centre_buffers`. Add an `assert` statement to check whether the two data frames are in the same CRS.

In [3]:
# ADD YOUR OWN CODE HERE

# Load the buffers computed in problem 2 into a GeoDataFrame called shopping_centre_buffers
shopping_centre_buffers = gpd.read_file(DATA_DIRECTORY / "shopping_centres.gpkg", layer="buffers")


# Add an assert statement to check whether the two data frames are in the same CRS.
assert shopping_centre_buffers.crs == population_grid.crs, "CRS are not identical"


In [4]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
assert isinstance(shopping_centre_buffers, geopandas.GeoDataFrame)
assert shopping_centre_buffers.geometry.geom_type.unique() == ["Polygon"]
assert shopping_centre_buffers.crs == pyproj.CRS("EPSG:3879")


---

### b) Carry out a *spatial join* between the `population_grid` and the `shopping_centre_buffers`

Join the shopping centre’s `id` column (and others, if you want) to the population grid data frame, for all population grid cells that are **within** the buffer area of each shopping centre. [Use a *join-type* that retains only rows from both input data frames for which the geometric predicate is true](https://geopandas.org/en/stable/gallery/spatial_joins.html#Types-of-spatial-joins). 


In [5]:
# ADD YOUR OWN CODE HERE
shopping_centre_buffers

,address,id,name,addr,geometry
0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","POLYGON ((25506098.602 6677662.109, 25506091.3..."
1,"Jungle Juice Bar, 14-20, Mannerheimintie, Kesk...",2,Forum,"Mannerheimintie 14–20, 00100 Helsinki, Finland","POLYGON ((25498047.460 6672895.008, 25498040.2..."
2,"Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...",3,Iso-omena,"Piispansilta 11, 02230 Espoo, Finland","POLYGON ((25486970.435 6672070.967, 25486963.2..."
3,"Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...",4,Sello,"Leppävaarankatu 3-9, 02600 Espoo, Finland","POLYGON ((25490991.076 6678322.265, 25490983.8..."
4,"Stockmann, 3, Vantaanportinkatu, Vantaanportti...",5,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa, Finland","POLYGON ((25499443.932 6686656.982, 25499436.7..."
5,"Silta, 5, Hermannin rantatie, Suvilahti, Kalas...",6,REDI,"Hermannin rantatie 5, 00580 Helsinki, Finland","POLYGON ((25500337.156 6674909.983, 25500329.9..."
6,"Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...",7,Tripla,"Mall of Tripla, Fredikanterassi 1, 00520 Helsi...","POLYGON ((25497645.658 6676164.227, 25497638.4..."


In [6]:
population_grid = population_grid[["asukkaita", "geometry"]]
population_grid = population_grid.rename(columns={"asukkaita": "population"})
population_grid

,population,geometry
0,5,"POLYGON ((25472499.995 6685998.998, 25472499.9..."
1,8,"POLYGON ((25472499.995 6684249.004, 25472499.9..."
2,5,"POLYGON ((25472499.995 6683999.005, 25472499.9..."
3,13,"POLYGON ((25472499.995 6682998.998, 25472499.9..."
4,5,"POLYGON ((25472749.993 6690249.003, 25472749.9..."
...,...,...
5832,7,"POLYGON ((25513249.999 6686248.997, 25513249.9..."
5833,6,"POLYGON ((25513249.999 6685998.998, 25513249.9..."
5834,14,"POLYGON ((25513249.999 6685748.999, 25513249.9..."
5835,12,"POLYGON ((25513249.999 6685499.000, 25513249.9..."


In [11]:
population_within_buffers = population_grid.sjoin(
    shopping_centre_buffers,
    how="right",
    predicate="within"
)

population_within_buffers

,index_left,population,address,id,name,addr,geometry
0,5232,63,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","POLYGON ((25506098.602 6677662.109, 25506091.3..."
0,5231,43,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","POLYGON ((25506098.602 6677662.109, 25506091.3..."
0,5230,80,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","POLYGON ((25506098.602 6677662.109, 25506091.3..."
0,5229,319,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","POLYGON ((25506098.602 6677662.109, 25506091.3..."
0,5344,202,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","POLYGON ((25506098.602 6677662.109, 25506091.3..."
...,...,...,...,...,...,...,...
6,3388,17,"Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...",7,Tripla,"Mall of Tripla, Fredikanterassi 1, 00520 Helsi...","POLYGON ((25497645.658 6676164.227, 25497638.4..."
6,3235,162,"Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...",7,Tripla,"Mall of Tripla, Fredikanterassi 1, 00520 Helsi...","POLYGON ((25497645.658 6676164.227, 25497638.4..."
6,3573,395,"Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...",7,Tripla,"Mall of Tripla, Fredikanterassi 1, 00520 Helsi...","POLYGON ((25497645.658 6676164.227, 25497638.4..."
6,3572,963,"Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...",7,Tripla,"Mall of Tripla, Fredikanterassi 1, 00520 Helsi...","POLYGON ((25497645.658 6676164.227, 25497638.4..."



---

### c) Compute the population sum around shopping centres

Group the resulting (joint) data frame by shopping centre (`id` or `name`), and calculate the `sum()` of the population living inside the 1.5 km radius around them.

Print the results, for instance, in the form "12345 people live within 1.5 km from REDI".

In [21]:
# ADD YOUR OWN CODE HERE

# Group the DataFrame by 'id' and calculate the sum of 'population'
supermarket_catchment_area = population_within_buffers.groupby("name")["population"].sum()

# Iterate over the grouped DataFrame and print the results
for name, population_sum in supermarket_catchment_area.items():
    print(f"{population_sum} people live within 1.5 km from {name}")

56099 people live within 1.5 km from Forum
26698 people live within 1.5 km from Iso-omena
20889 people live within 1.5 km from Itis
10907 people live within 1.5 km from Jumbo
27801 people live within 1.5 km from REDI
22329 people live within 1.5 km from Sello
24336 people live within 1.5 km from Tripla



---

### d) Reflection

Good job! You are almost done with this week’s exercise. Please quickly answer the following short questions:
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

Add your answers in a new *Markdown* cell below: